In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Load dataset (replace with actual dataset path)
data = pd.read_csv('temperature_data.csv', parse_dates=['date'], index_col='date')

# Preprocessing
def create_sequences(data, seq_length):
    sequences, targets = [], []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i+seq_length])
        targets.append(data[i+seq_length])
    return np.array(sequences), np.array(targets)

# Normalize data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# Create sequences
SEQ_LENGTH = 10
X, y = create_sequences(data_scaled, SEQ_LENGTH)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build LSTM model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(SEQ_LENGTH, X.shape[2])),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# Train model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Predict
def predict_temperature(input_seq):
    input_seq = scaler.transform(input_seq)
    input_seq = np.array(input_seq).reshape(1, SEQ_LENGTH, X.shape[2])
    predicted = model.predict(input_seq)
    return scaler.inverse_transform(predicted.reshape(-1, 1))[0, 0]

# Example usage:
# last_10_days = data_scaled[-SEQ_LENGTH:]
# next_temp = predict_temperature(last_10_days)
# print(f'Predicted Temperature: {next_temp}')
